# BGP DCN Configuration Information

Once the BGP DCN configuration has been added, either through the provided book (BGP_ClosBuilder) or manually, this book will provide you with commands to view information about the configuration of individual nodes in the topology.

## Topology Information

| Variable | Use |
| --- | --- |
| SLICE_NAME    | Name of the slice you want to access and observe |
| NETWORK_NODE_PREFIXES    | The naming prefix(es) for BGP-speaking nodes |
| COMPUTE_NODE_PREFIXES | The naming prefix(es) for compute/non-BGP-speaking nodes |

In [ ]:
SLICE_NAME = "1pod_test_bgp"
NETWORK_NODE_PREFIXES = "T,S,L"
COMPUTE_NODE_PREFIXES = "C"

## Access Slice Resources

The FabOrchestrator class is used to grab information. This is a class that wraps around the underlying FabLib to make certain actions easier to run.

In [ ]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)
    
except Exception as e:
    print(f"Exception: {e}")

## Routing

Anything involing how Linux kernel routing table.

In [ ]:
# Grab the IP Routing table from each BGP node.
print("ROUTING TABLES ON BGP NODES\n")

routingTableCommand = "ip route"
manager.executeCommandsParallel(routingTableCommand, prefixList=NETWORK_NODE_PREFIXES)

In [ ]:
# Grab the IP Routing table from each compute node.
print("ROUTING TABLES ON COMPUTE NODES\n")

routingTableCommand = "ip route"
manager.executeCommandsParallel(routingTableCommand, prefixList=COMPUTE_NODE_PREFIXES)

In [ ]:
# Grab the interface status from each node (regardless of the type of node).
print("NETWORK INTERFACE STATUS\n")

intfStatusCmd = "ip address | grep eth"
manager.executeCommandsParallel(intfStatusCmd, prefixList=NETWORK_NODE_PREFIXES)

In [ ]:
# Find any interfaces that are set to down
print("CHECK IF INTERFACES ARE DOWN\n")

intfStatusCmd = "ip address | grep DOWN"
manager.executeCommandsParallel(intfStatusCmd, prefixList=NETWORK_NODE_PREFIXES)

## Configuration

Anything involving how FRR or a FABRIC book configures the BGP daemon on the slices.

In [ ]:
outputConfigCommand = 'sudo vtysh -c "show run"'
manager.executeCommandsParallel(outputConfigCommand, prefixList=NETWORK_NODE_PREFIXES)

In [ ]:
outputConfigCommand = 'tmux kill-session -t bgp'
manager.executeCommandsParallel(outputConfigCommand, prefixList=NETWORK_NODE_PREFIXES)

In [ ]:
outputConfigCommand = 'sudo pkill tshark'
manager.executeCommandsParallel(outputConfigCommand, prefixList=NETWORK_NODE_PREFIXES)

In [ ]:
outputConfigCommand = 'tmux ls'
manager.executeCommandsParallel(outputConfigCommand, prefixList=NETWORK_NODE_PREFIXES)

In [ ]:
outputConfigCommand = 'ps aux | grep tshark'
manager.executeCommandsParallel(outputConfigCommand, prefixList=NETWORK_NODE_PREFIXES)

In [ ]:
manager.uploadDirectoryParallel("/home/fabric/work/custom/FABRIC-Automation/remote_scripts/bgp_scripts", prefixList=NETWORK_NODE_PREFIXES)

In [ ]:
outputConfigCommand = 'chmod +x /home/rocky/bgp_scripts/*.sh ; chmod +x /home/rocky/bgp_scripts/*.py'
manager.executeCommandsParallel(outputConfigCommand, prefixList=NETWORK_NODE_PREFIXES)

## Logging

The Clos Builder script logs all BGP UPDATE events to /var/log/bgpd.log

In [ ]:
# Grab the IP Routing table from each BGP node.
print("LOG ENTRIES\n")

routingTableEntry = "-- withdrawn"
routingTableCommand = f"sudo cat /var/log/frr/bgpd.log | grep {routingTableEntry}"
manager.executeCommandsParallel(routingTableCommand, prefixList=NETWORK_NODE_PREFIXES)